In [1]:
import tensorflow as tf
import keras
import numpy as np

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

tf.__version__

Using TensorFlow backend.


'1.5.0'

## GPU and CPU settings
If GPU is not available, comment out the bottom block.

In [2]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

## Define input and output

In [3]:
modelname = "knuckle1000"

input_path = "./"
input_file = modelname + ".h5"

output_path = "./"
output_file = modelname + ".pb"

output_node_prefix = "output_node"

In [4]:
K.set_learning_phase(0)
net_model = load_model(input_path + input_file)

num_output = 1
pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = output_node_prefix+str(i)
    pred[i] = tf.identity(net_model.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)
output_node_prefix = pred_node_names[0]

output nodes names are:  ['output_node0']


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [5]:
sess = K.get_session()

## Export model

In [6]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)

graph_io.write_graph(constant_graph, output_path, output_file, as_text=False)

print('Saved the freezed graph at: ', (output_path + output_file))

INFO:tensorflow:Froze 8 variables.
Converted 8 variables to const ops.
Saved the freezed graph at:  ./knuckle1000.pb


## Show input and output node

In [7]:
g = tf.GraphDef()
g.ParseFromString(open(output_path + output_file, "rb").read())
s = ""
for n in g.node:
    s =s + str(n.name) + "\n"

print(s)

conv2d_7_input
conv2d_7/kernel
conv2d_7/kernel/read
conv2d_7/bias
conv2d_7/bias/read
conv2d_7/convolution
conv2d_7/BiasAdd
conv2d_7/Relu
conv2d_8/kernel
conv2d_8/kernel/read
conv2d_8/bias
conv2d_8/bias/read
conv2d_8/convolution
conv2d_8/BiasAdd
conv2d_8/Relu
max_pooling2d_4/MaxPool
dropout_4/Identity
flatten_4/Shape
flatten_4/strided_slice/stack
flatten_4/strided_slice/stack_1
flatten_4/strided_slice/stack_2
flatten_4/strided_slice
flatten_4/Const
flatten_4/Prod
flatten_4/stack/0
flatten_4/stack
flatten_4/Reshape
dense_7/kernel
dense_7/kernel/read
dense_7/bias
dense_7/bias/read
dense_7/MatMul
dense_7/BiasAdd
dense_7/Relu
dense_8/kernel
dense_8/kernel/read
dense_8/bias
dense_8/bias/read
dense_8/MatMul
dense_8/BiasAdd
dense_8/Softmax
output_node0

